# VoronoiSite example

In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
from pymatgen.core import Lattice, Structure
import numpy as np

from site_analysis.atom import Atom
from site_analysis.trajectory import Trajectory
from site_analysis.voronoi_site import VoronoiSite
from site_analysis.tools import get_vertex_indices

from collections import Counter
import tqdm

Load a `POSCAR` file where every octahedral site is occupied by a Na atom.

In [2]:
all_na_structure = Poscar.from_file('na_sn_all_na_ext.POSCAR.vasp').structure
vertex_species = 'S'
centre_species = 'Na'

Create a series of pymatgen Structures using the `Structure.from_spacegroup()` method, that each only contain the NaX sites, using the coordinates from Ramos _et al._ _Chem. Mater._ 2018.

In [3]:
sg = SpaceGroup('I41/acd:2')

In [4]:
lattice = all_na_structure.lattice
na1 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.25, 0.0, 0.125]])
na2 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.00, 0.0, 0.125]])
na3 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.25, 0.0]])
na4 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.0, 0.0]])
na5 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.75, 0.25, 0.0]])
na6 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.5, 0.75, 0.625]])
i2  = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.666, 0.1376, 0.05]])
na_structures = {'Na1': na1,
                 'Na2': na2,
                 'Na3': na3,
                 'Na4': na4,
                 'Na5': na5,
                 'Na6': na6, 
                 'i2': i2}

In [5]:
na1_sites = [VoronoiSite(s.frac_coords, label='Na1') for s in na1]
na2_sites = [VoronoiSite(s.frac_coords, label='Na2') for s in na2]
na3_sites = [VoronoiSite(s.frac_coords, label='Na3') for s in na3]
na4_sites = [VoronoiSite(s.frac_coords, label='Na4') for s in na4]
na5_sites = [VoronoiSite(s.frac_coords, label='Na5') for s in na5]
na6_sites = [VoronoiSite(s.frac_coords, label='Na6') for s in na6]
i2_sites  = [VoronoiSite(s.frac_coords, label='i2') for s in i2]
sites = na1_sites + na2_sites + na3_sites + na4_sites + na5_sites + na6_sites + i2_sites

In [6]:
structure = Poscar.from_file('POSCAR').structure
# create Polyhedron objects
# create Atom objects
# Note: index is the atom index in the pymatgen structures we are going to process.
# In this example, Na are the first species, so we count from 0.
atoms = [Atom(index=i, species_string=centre_species) for i, site in enumerate(structure) if site.species_string == 'Na']
trajectory = Trajectory(sites, atoms)

In [7]:
print(trajectory.site_labels())

['Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2']


In [8]:
trajectory.append_timestep(structure)

The occupations of each site are stored as a list of lists, as each site can have zero, one, or multiple atoms occupying it.

In [9]:
print(trajectory.sites_trajectory[0])

[[83], [81], [84], [4], [74], [8], [77], [78], [82], [80], [], [87], [15], [72], [76], [79], [0], [75], [73], [11], [3], [13], [], [7], [14], [23], [20], [26], [16], [27], [18], [22], [1], [9], [6], [10], [2], [12], [5], [], [], [24], [19], [25], [85], [86], [17], [21], [56], [32], [28], [33], [46], [31], [29], [64], [], [50], [35], [39], [61], [37], [36], [38], [34], [66], [63], [69], [59], [71], [60], [], [67], [68], [70], [65], [57], [62], [58], [30], [47], [43], [45], [41], [55], [], [52], [49], [], [42], [44], [40], [54], [51], [53], [48], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


Rough example for collecting only occupied sites, and counting their site types:

In [10]:
c = Counter()
for site in trajectory.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
c

Counter({'Na1': 15,
         'Na2': 29,
         'Na3': 15,
         'Na4': 15,
         'Na5': 14,
         'Na6': 0,
         'i2': 0})

vs. all sites:

In [11]:
c_sites = Counter(trajectory.site_labels())
c_sites

Counter({'Na1': 16,
         'Na2': 32,
         'Na3': 16,
         'Na4': 16,
         'Na5': 16,
         'Na6': 8,
         'i2': 32})

In [12]:
trajectory.reset()

xdatcar = Xdatcar('XDATCAR_Sn')

trajectory.trajectory_from_structures( xdatcar.structures, progress='notebook')

  0%|          | 0/300 [00:00<?, ? steps/s]

In [13]:
n_timesteps = len(trajectory.timesteps)
c_sites = Counter(trajectory.site_labels())
c = Counter()
p_occ = {}
for site in trajectory.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for k, v in c.items():
    p_occ[k] = v / c_sites[k] / n_timesteps
p_occ

{'Na1': 0.93375,
 'Na2': 0.895625,
 'Na3': 0.915,
 'Na4': 0.9427083333333334,
 'Na5': 0.9120833333333334,
 'Na6': 0.0,
 'i2': 0.0026041666666666665}

In [14]:
# check total average occupation = 88 atoms
for k,v in c.items():
    print( k, p_occ[k]*c_sites[k])
print( sum( [ p_occ[k] * c_sites[k] for k, v in c.items()]))

Na1 14.94
Na2 28.66
Na3 14.64
Na4 15.083333333333334
Na5 14.593333333333334
Na6 0.0
i2 0.08333333333333333
88.0
